# Exploring Hacker News Posts

In this project, we'll work with a data set of submissions to popular technology site [Hacker News](https://news.ycombinator.com/).

![hacker news logo](https://s3.amazonaws.com/dq-content/354/hacker_news.jpg "hacker news logo")

Hacker news is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as 'posts') are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

You can find the dataset [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submisstions. Below are the descriptions of the columns:
- <code style="color:red">id</code> - The unique identifier from Hacker News for the post
- <code style="color:red">title</code> - The title of the post
- <code style="color:red">url</code> - The URL that the post links to, if the post has a URL
- <code style="color:red">num_points</code> - The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- <code style="color:red">num_comments</code> - The number of comments that were made on the post
- <code style="color:red">author</code> - The username of the person who submitted the post
- <code style="color:red">created_at</code> - The date and time at which the post was submitted

We're speciffically interested in posts whose titles begin with either <code style="color:red">Ask HN</code> or <code style="color:red">Show HN</code>. Users submit <code style="color:red">Ask HN</code> posts to ask the Hacker News community a specific question. Likewise, users submit <code style="color:red">Show HN</code> posts to show the Hacker News community a project, product, or just generally something interesting.

We'll compare these two types of posts to determine the followin:
- Do <code style="color:red">Ask HN</code> or <code style="color:red">Show HN</code> receive more comments on average
- Do posts created at a certain time receive more comments on average?


# Introduction

Let's start by importing the libraries we need and reading the data set into a list of lists. We'll also extract the headers to their own list.

In [20]:
import csv

hn = open('hacker_news.csv', encoding='utf-8')
hn = csv.reader(hn)
hn = list(hn)

headers = hn[1]
hn = hn[1:]

print(headers, '\n')

hn[:5]



['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'] 



[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]

# Filtering Data

Since we're only concerned with posts beginning with <code style="color:red">Ask HN</code> or <code style="color:red">Show HN</code>, we'll create new lists of lists containing just the data for those titles.

In [21]:
ask_posts = []
show_posts = []
other_posts = []

for post in hn:
    title = post[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(post)
    elif title.lower().startswith('show hn'):
        show_posts.append(post)
    else:
        other_posts.append(post)
        
print("ask_posts has {} posts".format(len(ask_posts)))        
print("show_posts has {} posts".format(len(show_posts)))        
print("other_posts has {} posts".format(len(other_posts)))        
    

ask_posts has 1744 posts
show_posts has 1162 posts
other_posts has 17194 posts


# Calculate average number of comments for Ask and Show posts

Let's determine if ask posts or show posts receive more comments on average.



In [22]:
total_ask_comments = 0
for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments
    
avg_ask_comments = total_ask_comments / len(ask_posts)    
print("Ask posts have on average {:.2f} comments".format(avg_ask_comments))

total_show_comments = 0
for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments / len(show_posts)    
print("Show posts have on average {:.2f} comments".format(avg_show_comments))

Ask posts have on average 14.04 comments
Show posts have on average 10.32 comments


As we can see, ask posts have on average 14.04 comments, while show posts have on average of 10.32 comments. Ask posts receive more comments than show posts.

For the rest of this analysis we'll focus just on the ask posts.

# Average number of comments Ask posts receive per hour created

We'll now determine if ask posts created at a certain time are more likely to attract comments. We will do the following steps:
1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received
2. Calculate the average number of comments ask posts receive by hour created.

In [44]:
# calculate the amount of ask posts per hour of the day 
# and the number of comments received

import datetime as dt

result_list = []
for post in ask_posts:
    created_at = post[6]
    num_comments = int(post[4])
    result_list.append([created_at, num_comments])
    #print(post[6])
    
counts_by_hour = {}
comments_by_hour = {}
for post in result_list:
    date = dt.datetime.strptime(post[0], "%m/%d/%Y %H:%M")
    hour = date.strftime("%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = post[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += post[1]

In [53]:
# calculate the average number of comments ask posts receive by hour created

avg_by_hour = []
for hour in counts_by_hour:
    average = comments_by_hour[hour] / counts_by_hour[hour]
    avg_by_hour.append([hour, average])
    
avg_by_hour


[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

We have the results we need, but this format makes it hard to identify the hours with the highest values.

# Sorting and printing the highest values

In [57]:
swap_avg_by_hour = []
for hour in avg_by_hour:
    swap_avg_by_hour.append([hour[1], hour[0]])
    
swap_avg_by_hour

[[5.5777777777777775, '09'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [16.796296296296298, '16'],
 [7.985294117647059, '23'],
 [9.41095890410959, '12'],
 [11.46, '17'],
 [38.5948275862069, '15'],
 [16.009174311926607, '21'],
 [21.525, '20'],
 [23.810344827586206, '02'],
 [13.20183486238532, '18'],
 [7.796296296296297, '03'],
 [10.08695652173913, '05'],
 [10.8, '19'],
 [11.383333333333333, '01'],
 [6.746478873239437, '22'],
 [10.25, '08'],
 [7.170212765957447, '04'],
 [8.127272727272727, '00'],
 [9.022727272727273, '06'],
 [7.852941176470588, '07'],
 [11.051724137931034, '11']]

In [68]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print("Top 5 Hours for Ask Posts Comments")

for el in sorted_swap[:5]:
    hour_datetime = dt.datetime.strptime(el[1], "%H")
    hour_string = hour_datetime.strftime("%H:%M")
    #print(hour_string)
    avg_string = "{}: {:.2f} average comments per post".format(hour_string, el[0])
    print(avg_string)

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

# Conclusion

In this project, we analyzed as posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as asp post and created between 15:00 and 16:00 est.

However, it should be noted that the dataset we analyzed excludes posts without any comments. Given that, it's more accurate to say that _of the posts that received comments_, ask posts received more comments on average and ask posts created between 15:00 and 16:00 est received the most comments on average.